In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import json
#Import package matplotlib for visualisation/plotting
# import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from sodapy import Socrata

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofnewyork.us",
                 "EXI398QZgUcSkIww5h9tF5v0u",
                 username="ly.nguyen@ucdconnect.ie",
                 password="mS.sMHVPLQn5*tU")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("w7w3-xahh", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
results_df.shape

(281244, 32)

In [4]:
results_df.head(5)

,license_nbr,license_type,lic_expir_dd,license_status,license_creation_date,industry,business_name,business_name_2,address_building,address_street_name,...,census_tract,longitude,latitude,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,address_street_name_2
0,0967332-DCA,Business,2017-02-28T00:00:00.000,Inactive,2010-02-04T00:00:00.000,Home Improvement Contractor,"BARBARINO, JOHN JR.",JOHN BARBARINO JR_HOME IMPROVEMENT,239,MEDFORD CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2103411-DCA,Individual,2024-03-31T00:00:00.000,Active,2022-01-11T00:00:00.000,Sightseeing Guide,"Nieciak, Joseph",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2060087-DCA,Individual,2024-03-31T00:00:00.000,Active,2017-10-31T00:00:00.000,Sightseeing Guide,"Arcaro, Tyler",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2078026-DCA,Individual,2024-03-31T00:00:00.000,Active,2018-09-11T00:00:00.000,Sightseeing Guide,"HESSE, DAPHNEE",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2057801-DCA,Individual,2019-02-28T00:00:00.000,Inactive,2017-09-01T00:00:00.000,Home Improvement Salesperson,"ESCOBAR, KEVIN",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
results_df.dtypes

license_nbr                    object
license_type                   object
lic_expir_dd                   object
license_status                 object
license_creation_date          object
industry                       object
business_name                  object
business_name_2                object
address_building               object
address_street_name            object
address_city                   object
address_state                  object
address_zip                    object
contact_phone                  object
address_borough                object
detail_2                       object
detail                         object
community_board                object
council_district               object
bin                            object
bbl                            object
nta                            object
census_tract                   object
longitude                      object
latitude                       object
location                       object
:@computed_r

In [7]:
# Convert 'License Expiration Date' to datetime
results_df['lic_expir_dd'] = pd.to_datetime(results_df['lic_expir_dd'])
results_df['license_creation_date'] = pd.to_datetime(results_df['license_creation_date'])
results_df.dtypes

license_nbr                            object
license_type                           object
lic_expir_dd                   datetime64[ns]
license_status                         object
license_creation_date          datetime64[ns]
industry                               object
business_name                          object
business_name_2                        object
address_building                       object
address_street_name                    object
address_city                           object
address_state                          object
address_zip                            object
contact_phone                          object
address_borough                        object
detail_2                               object
detail                                 object
community_board                        object
council_district                       object
bin                                    object
bbl                                    object
nta                               

In [8]:
# Filter rows
old = results_df[results_df['lic_expir_dd'] < '2022-01-01']

# Check
results_df = results_df.drop(old.index)

In [9]:
results_df.shape

(77635, 32)

# drop the rows of Borough are out of New York.

In [10]:
# drop the rows of Borough are out of New York
results_df = results_df[results_df['address_borough'] != "Outside NYC"]
results_df.shape

(71720, 32)

# drop the rows of state are out of New York.

In [12]:
# check the rows of state are out of New York
results_df = results_df[results_df['address_state'] == "NY"]
results_df.shape

(68714, 32)

# drop the rows which zip code are out of NYC

In [13]:
# Convert the data type to int
results_df['address_zip'] = results_df['address_zip'].astype(int)

In [16]:
# check the rows of zip code are out of New York
results_df = results_df[(results_df['address_zip'] >= 10001) & (results_df['address_zip'] <= 11697)]

results_df.shape

(66750, 32)

# use zip code to match the rows without Longitude and Latitude.

In [19]:
# check the rows without Longitude and Latitude.
without_long_lat = results_df[results_df['longitude'].isnull() | results_df['latitude'].isnull()]
without_long_lat.shape

(25907, 32)

# don't run the code below, the function run whole night because the package server rate limits, i have already save output csv, called add_long_lat.csv. we will read and load this csv soon.

In [22]:
def geolocate(zip):
    location = geolocator.geocode(zip)
    return location.latitude, location.longitude

without_long_lat = results_df[results_df['latitude'].isna() & results_df['longitude'].isna()].index

results_df.loc[without_long_lat, 'latitude'] = results_df.loc[without_long_lat, 'address_zip'].apply(lambda x: geolocate(x)[0])
results_df.loc[without_long_lat, 'longitude'] = results_df.loc[without_long_lat, 'address_zip'].apply(lambda x: geolocate(x)[1])


NameError: name 'geolocator' is not defined

In [253]:
df

,Industry,Address State,Address Borough,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP
1,Sightseeing Guide,NY,NaN,-73.996614,40.602054,2024-03-31,01/11/2022,11214
2,Sightseeing Guide,NY,NaN,-73.953741,40.776551,2024-03-31,10/31/2017,10028
7,Sightseeing Guide,NY,NaN,-73.981113,40.776917,2024-03-31,05/23/2000,10023
9,Ticket Seller,NY,NaN,-73.918217,40.807720,2023-07-31,07/15/2022,10454
12,Ticket Seller,NY,NaN,-74.088153,40.609754,2023-07-31,05/14/2019,10304
...,...,...,...,...,...,...,...,...
280676,Locksmith,NY,NaN,-73.817992,40.598996,2025-05-31,03/19/2013,11693
280677,Home Improvement Contractor,NY,Bronx,-73.871939,40.835076,2025-02-28,08/16/2022,10472
280678,Home Improvement Contractor,NY,Queens,-73.875185,40.722776,2025-02-28,01/31/2022,11379
280679,Electronics Store,NY,Queens,-73.766298,40.760707,2024-12-31,12/08/2009,11361


In [359]:
# df.to_csv("add_long_lat.csv", index=False)

# run at here!!!!

In [396]:
# Reading from a csv file, into a data frame
df = pd.read_csv('basic_add_long_lat.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [397]:
df

,Industry,Address State,Address Borough,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP
0,Sightseeing Guide,NY,NaN,-73.996614,40.602054,2024-03-31,01/11/2022,11214
1,Sightseeing Guide,NY,NaN,-73.953741,40.776551,2024-03-31,10/31/2017,10028
2,Sightseeing Guide,NY,NaN,-73.981113,40.776917,2024-03-31,05/23/2000,10023
3,Ticket Seller,NY,NaN,-73.918217,40.807720,2023-07-31,07/15/2022,10454
4,Ticket Seller,NY,NaN,-74.088153,40.609754,2023-07-31,05/14/2019,10304
...,...,...,...,...,...,...,...,...
38820,Locksmith,NY,NaN,-73.817992,40.598996,2025-05-31,03/19/2013,11693
38821,Home Improvement Contractor,NY,Bronx,-73.871939,40.835076,2025-02-28,08/16/2022,10472
38822,Home Improvement Contractor,NY,Queens,-73.875185,40.722776,2025-02-28,01/31/2022,11379
38823,Electronics Store,NY,Queens,-73.766298,40.760707,2024-12-31,12/08/2009,11361


In [398]:
# check the data type
df.dtypes

Industry                    object
Address State               object
Address Borough             object
Longitude                  float64
Latitude                   float64
License Expiration Date     object
License Creation Date       object
Address ZIP                  int64
dtype: object

In [399]:
# check null value
df.isnull().sum()

Industry                      0
Address State                 0
Address Borough            8991
Longitude                     0
Latitude                      0
License Expiration Date       4
License Creation Date         0
Address ZIP                   0
dtype: int64

In [400]:
df = df.dropna(subset=['License Expiration Date'])

In [401]:
df

,Industry,Address State,Address Borough,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP
0,Sightseeing Guide,NY,NaN,-73.996614,40.602054,2024-03-31,01/11/2022,11214
1,Sightseeing Guide,NY,NaN,-73.953741,40.776551,2024-03-31,10/31/2017,10028
2,Sightseeing Guide,NY,NaN,-73.981113,40.776917,2024-03-31,05/23/2000,10023
3,Ticket Seller,NY,NaN,-73.918217,40.807720,2023-07-31,07/15/2022,10454
4,Ticket Seller,NY,NaN,-74.088153,40.609754,2023-07-31,05/14/2019,10304
...,...,...,...,...,...,...,...,...
38820,Locksmith,NY,NaN,-73.817992,40.598996,2025-05-31,03/19/2013,11693
38821,Home Improvement Contractor,NY,Bronx,-73.871939,40.835076,2025-02-28,08/16/2022,10472
38822,Home Improvement Contractor,NY,Queens,-73.875185,40.722776,2025-02-28,01/31/2022,11379
38823,Electronics Store,NY,Queens,-73.766298,40.760707,2024-12-31,12/08/2009,11361


In [402]:
# check null value
df.isnull().sum()

Industry                      0
Address State                 0
Address Borough            8990
Longitude                     0
Latitude                      0
License Expiration Date       0
License Creation Date         0
Address ZIP                   0
dtype: int64

In [403]:
df['Industry'] = df['Industry'].astype('category')
df['Address State'] = df['Address State'].astype('category')
df['Address Borough'] = df['Address Borough'].astype('category')

/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/2719037329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Industry'] = df['Industry'].astype('category')
/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/2719037329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Address State'] = df['Address State'].astype('category')
/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/2719037329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [404]:
#update all columns of type 'category'
category_columns = df.select_dtypes(['category']).columns
category_columns

Index(['Industry', 'Address State', 'Address Borough'], dtype='object')

In [405]:
# show category date type
new_df_table_categ = df[category_columns].describe().T
new_df_table_categ

,count,unique,top,freq
Industry,38821,44,Home Improvement Contractor,8939
Address State,38821,1,NY,38821
Address Borough,29831,7,Queens,8752


In [406]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Longitude,38821.0,-69.492915,27.035518,-99.262496,-73.980828,-73.928436,-73.862583,126.98922
Latitude,38821.0,40.374275,2.956185,-6.187254,40.651692,40.723613,40.775202,59.83742
Address ZIP,38821.0,10823.526725,555.101925,10001.000000,10306.000000,11204.000000,11239.000000,11697.00000


In [407]:
print("Industry: ", df['Industry'].unique().tolist(), "\n")
print("Address State: ", df['Address State'].unique().tolist(), "\n")
print("Address Borough: ", df['Address Borough'].unique().tolist(), "\n")

Industry:  ['Sightseeing Guide', 'Ticket Seller', 'Locksmith', 'Electronic & Appliance Service', 'General Vendor', 'Secondhand Dealer - General', 'Employment Agency', 'Electronic Cigarette Dealer', 'Pedicab Driver', 'Newsstand', 'Tobacco Retail Dealer', 'Garage', 'Stoop Line Stand', 'Tow Truck Driver', 'Laundries', 'Electronics Store', 'Process Server Individual', 'Tow Truck Company', 'Pawnbroker', 'Home Improvement Contractor', 'Process Serving Agency', 'Horse Drawn Driver', 'Car Wash', 'Scrap Metal Processor', 'Construction Labor Provider', 'Secondhand Dealer - Auto', 'Third Party Food Delivery', 'Garage and Parking Lot', 'Dealer In Products', 'Sidewalk Cafe', 'Horse Drawn Cab Owner', 'Parking Lot', 'Storage Warehouse', 'Bingo Game Operator', 'Games of Chance', 'Commercial Lessor', 'Debt Collection Agency', 'Pedicab Business', 'Scale Dealer Repairer', 'Sightseeing Bus', 'Booting Company', 'Ticket Seller Business', 'Locksmith Apprentice', 'General Vendor Distributor'] 

Address State:

# keep the useful features: Industry, Longitude and Latitude...

In [408]:
df = df[['Industry', 'Longitude', 'Latitude', 'License Expiration Date', 'License Creation Date', 'Address ZIP']]
df

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP
0,Sightseeing Guide,-73.996614,40.602054,2024-03-31,01/11/2022,11214
1,Sightseeing Guide,-73.953741,40.776551,2024-03-31,10/31/2017,10028
2,Sightseeing Guide,-73.981113,40.776917,2024-03-31,05/23/2000,10023
3,Ticket Seller,-73.918217,40.807720,2023-07-31,07/15/2022,10454
4,Ticket Seller,-74.088153,40.609754,2023-07-31,05/14/2019,10304
...,...,...,...,...,...,...
38820,Locksmith,-73.817992,40.598996,2025-05-31,03/19/2013,11693
38821,Home Improvement Contractor,-73.871939,40.835076,2025-02-28,08/16/2022,10472
38822,Home Improvement Contractor,-73.875185,40.722776,2025-02-28,01/31/2022,11379
38823,Electronics Store,-73.766298,40.760707,2024-12-31,12/08/2009,11361


In [409]:
# check null value
df.isnull().sum()

Industry                   0
Longitude                  0
Latitude                   0
License Expiration Date    0
License Creation Date      0
Address ZIP                0
dtype: int64

# do the point-in-polygon operation to match longitude and latitude points to specific taxi zones 

In [410]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [411]:
# change 'Longitude' and 'Latitude' into GeoDataFrame
df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
df['Coordinates'] = df['Coordinates'].apply(Point)

gdf_business = gpd.GeoDataFrame(df, geometry='Coordinates')

/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/593872596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/593872596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Coordinates'] = df['Coordinates'].apply(Point)


In [412]:
gdf_business

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,Coordinates
0,Sightseeing Guide,-73.996614,40.602054,2024-03-31,01/11/2022,11214,POINT (-73.99661 40.60205)
1,Sightseeing Guide,-73.953741,40.776551,2024-03-31,10/31/2017,10028,POINT (-73.95374 40.77655)
2,Sightseeing Guide,-73.981113,40.776917,2024-03-31,05/23/2000,10023,POINT (-73.98111 40.77692)
3,Ticket Seller,-73.918217,40.807720,2023-07-31,07/15/2022,10454,POINT (-73.91822 40.80772)
4,Ticket Seller,-74.088153,40.609754,2023-07-31,05/14/2019,10304,POINT (-74.08815 40.60975)
...,...,...,...,...,...,...,...
38820,Locksmith,-73.817992,40.598996,2025-05-31,03/19/2013,11693,POINT (-73.81799 40.59900)
38821,Home Improvement Contractor,-73.871939,40.835076,2025-02-28,08/16/2022,10472,POINT (-73.87194 40.83508)
38822,Home Improvement Contractor,-73.875185,40.722776,2025-02-28,01/31/2022,11379,POINT (-73.87518 40.72278)
38823,Electronics Store,-73.766298,40.760707,2024-12-31,12/08/2009,11361,POINT (-73.76630 40.76071)


In [413]:
# Read the geojson file
taxi_zone = gpd.read_file('NYC Taxi Zones.geojson')
taxi_zone

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [414]:
gdf_taxi_zone = gpd.GeoDataFrame(taxi_zone, geometry='geometry')
gdf_taxi_zone

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [415]:
# Ensure both GeoDataFrames are using the same CRS
gdf_business = gdf_business.set_crs("EPSG:4326")
gdf_taxi_zone = gdf_taxi_zone.set_crs("EPSG:4326")

# Perform spatial join with the 'predicate' parameter
result = gpd.sjoin(gdf_business, gdf_taxi_zone, how="inner", predicate='intersects')

In [416]:
result

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,Coordinates,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
0,Sightseeing Guide,-73.996614,40.602054,2024-03-31,01/11/2022,11214,POINT (-73.99661 40.60205),18,0.000472062684608,22,0.126170229196,22,Bensonhurst West,Brooklyn
31,General Vendor,-73.996614,40.602054,2023-09-30,07/15/2022,11214,POINT (-73.99661 40.60205),18,0.000472062684608,22,0.126170229196,22,Bensonhurst West,Brooklyn
73,Stoop Line Stand,-73.993377,40.601692,2024-03-31,07/18/2022,11214,POINT (-73.99338 40.60169),18,0.000472062684608,22,0.126170229196,22,Bensonhurst West,Brooklyn
140,Tobacco Retail Dealer,-73.999085,40.618822,2023-12-31,07/22/2020,11228,POINT (-73.99909 40.61882),18,0.000472062684608,22,0.126170229196,22,Bensonhurst West,Brooklyn
202,Stoop Line Stand,-73.993154,40.601555,2024-03-31,07/15/2022,11214,POINT (-73.99315 40.60155),18,0.000472062684608,22,0.126170229196,22,Bensonhurst West,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36608,Garage,-74.014434,40.703221,2025-03-31,03/28/1997,10004,POINT (-74.01443 40.70322),12,0.0000415116236727,12,0.0366613013579,12,Battery Park,Manhattan
30475,Secondhand Dealer - Auto,-73.911641,40.599915,2023-07-31,02/10/2005,11234,POINT (-73.91164 40.59992),153,0.00178968692433,154,0.22733157076,154,Marine Park/Floyd Bennett Field,Brooklyn
31933,Tobacco Retail Dealer,-73.788242,40.648244,2024-12-31,07/14/2011,11430,POINT (-73.78824 40.64824),136,0.00203830095472,132,0.24547851707,132,JFK Airport,Queens
32085,Tobacco Retail Dealer,-73.788242,40.648244,2024-12-31,07/14/2011,11430,POINT (-73.78824 40.64824),136,0.00203830095472,132,0.24547851707,132,JFK Airport,Queens


In [417]:
result = result[['Industry', 'Longitude', 'Latitude', 'License Expiration Date', 'License Creation Date', 'Address ZIP', "location_id", "borough" ]]
result

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
0,Sightseeing Guide,-73.996614,40.602054,2024-03-31,01/11/2022,11214,22,Brooklyn
31,General Vendor,-73.996614,40.602054,2023-09-30,07/15/2022,11214,22,Brooklyn
73,Stoop Line Stand,-73.993377,40.601692,2024-03-31,07/18/2022,11214,22,Brooklyn
140,Tobacco Retail Dealer,-73.999085,40.618822,2023-12-31,07/22/2020,11228,22,Brooklyn
202,Stoop Line Stand,-73.993154,40.601555,2024-03-31,07/15/2022,11214,22,Brooklyn
...,...,...,...,...,...,...,...,...
36608,Garage,-74.014434,40.703221,2025-03-31,03/28/1997,10004,12,Manhattan
30475,Secondhand Dealer - Auto,-73.911641,40.599915,2023-07-31,02/10/2005,11234,154,Brooklyn
31933,Tobacco Retail Dealer,-73.788242,40.648244,2024-12-31,07/14/2011,11430,132,Queens
32085,Tobacco Retail Dealer,-73.788242,40.648244,2024-12-31,07/14/2011,11430,132,Queens


In [418]:
# check null value
result.isnull().sum()

Industry                   0
Longitude                  0
Latitude                   0
License Expiration Date    0
License Creation Date      0
Address ZIP                0
location_id                0
borough                    0
dtype: int64

In [419]:
print("Industry: ", result['Industry'].unique().tolist(), "\n")
print("location_id: ", result['location_id'].unique().tolist(), "\n")
print("borough: ", result['borough'].unique().tolist(), "\n")

Industry:  ['Sightseeing Guide', 'General Vendor', 'Stoop Line Stand', 'Tobacco Retail Dealer', 'Electronic & Appliance Service', 'Electronic Cigarette Dealer', 'Pedicab Driver', 'Secondhand Dealer - General', 'Ticket Seller', 'Home Improvement Contractor', 'Process Server Individual', 'Tow Truck Driver', 'Locksmith', 'Electronics Store', 'Secondhand Dealer - Auto', 'Car Wash', 'Dealer In Products', 'Pedicab Business', 'Horse Drawn Driver', 'Tow Truck Company', 'Employment Agency', 'Parking Lot', 'Scrap Metal Processor', 'Newsstand', 'Garage', 'Garage and Parking Lot', 'Pawnbroker', 'Laundries', 'Storage Warehouse', 'Construction Labor Provider', 'Games of Chance', 'Scale Dealer Repairer', 'Process Serving Agency', 'Ticket Seller Business', 'Bingo Game Operator', 'Locksmith Apprentice', 'Third Party Food Delivery', 'Debt Collection Agency', 'Sightseeing Bus', 'Sidewalk Cafe', 'Horse Drawn Cab Owner', 'Booting Company', 'Commercial Lessor', 'General Vendor Distributor'] 

location_id:  

In [421]:
result['Industry'] = result['Industry'].cat.add_categories(['food_and_beverage'])

# do the assignment
conditions = [
    (result['Industry'] == 'Catering Establishment'), 
    (result['Industry'] == 'Sidewalk Cafe'), 
    (result['Industry'] == 'Third Party Food Delivery')
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'food_and_beverage'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/3455375388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['food_and_beverage'])


In [422]:
# check the rows with Food and Beverage type

food_and_beverage = result[result['Industry'] == 'food_and_beverage']

food_and_beverage

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
386,food_and_beverage,-73.959583,40.729699,2023-08-31,04/27/2022,11222,112,Brooklyn
7596,food_and_beverage,-73.828964,40.764799,2023-08-31,02/17/2022,11354,92,Queens
7197,food_and_beverage,-73.944016,40.683915,2022-12-15,07/10/2015,11216,17,Brooklyn
19256,food_and_beverage,-73.990154,40.751641,2025-08-31,01/25/2022,10123,100,Manhattan
633,food_and_beverage,-73.972588,40.785904,2022-12-15,01/03/2020,10024,239,Manhattan
10555,food_and_beverage,-73.992175,40.754946,2025-08-31,04/06/2023,10018,48,Manhattan
17013,food_and_beverage,-73.995399,40.631808,2023-08-31,06/27/2022,11219,26,Brooklyn
721,food_and_beverage,-73.949182,40.789308,2023-08-31,04/27/2022,10029,75,Manhattan
796,food_and_beverage,-73.982647,40.665598,2022-09-15,01/13/2020,11215,181,Brooklyn
804,food_and_beverage,-74.009878,40.721696,2022-12-15,01/03/2020,10013,231,Manhattan


In [423]:
result['Industry'] = result['Industry'].cat.add_categories(['entertainment_and_recreation'])

# do the assignment
conditions = [
    (result['Industry'] == 'Gaming Cafe'), 
    (result['Industry'] == 'Amusement Arcade'), 
    (result['Industry'] == 'Cabaret'),
    (result['Industry'] == 'Bingo Game Operator'),
    (result['Industry'] == 'Amusement Device Permanent'),
    (result['Industry'] == 'Amusement Device Portable'),
    (result['Industry'] == 'Amusement Device Temporary'),
    (result['Industry'] == 'Games of Chance'),
    (result['Industry'] == 'Sightseeing Bus'),
    (result['Industry'] == 'Sightseeing Guide'),
    (result['Industry'] == 'Pool or Billiard Room'),
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'entertainment_and_recreation'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/569252661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['entertainment_and_recreation'])


In [424]:
# check the rows with entertainment_and_recreation type

entertainment_and_recreation = result[result['Industry'] == 'entertainment_and_recreation']

entertainment_and_recreation

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
0,entertainment_and_recreation,-73.996614,40.602054,2024-03-31,01/11/2022,11214,22,Brooklyn
1920,entertainment_and_recreation,-73.985921,40.618792,2024-03-31,04/12/2011,11204,22,Brooklyn
1978,entertainment_and_recreation,-73.996614,40.602054,2024-03-31,11/29/2019,11214,22,Brooklyn
2181,entertainment_and_recreation,-73.985921,40.618792,2024-03-31,10/16/2013,11204,22,Brooklyn
3324,entertainment_and_recreation,-73.996624,40.602054,2024-03-31,09/03/2014,11214,22,Brooklyn
...,...,...,...,...,...,...,...,...
21337,entertainment_and_recreation,-73.949929,40.762033,2024-03-31,08/30/2021,10044,202,Manhattan
31737,entertainment_and_recreation,-73.949929,40.761928,2024-03-31,11/16/2022,10044,202,Manhattan
15481,entertainment_and_recreation,-73.914911,40.559734,2024-03-31,06/13/2022,11697,27,Queens
20864,entertainment_and_recreation,-73.787644,40.848603,2024-03-31,09/19/2008,10464,46,Bronx


In [425]:
result['Industry'] = result['Industry'].cat.add_categories(['parking_and_automotive_services'])

# do the assignment
conditions = [
    (result['Industry'] == 'Car Wash'), 
    (result['Industry'] == 'Booting Company'), 
    (result['Industry'] == 'Garage'),
    (result['Industry'] == 'Garage and Parking Lot'),
    (result['Industry'] == 'Parking Lot'),
    (result['Industry'] == 'Secondhand Dealer - Auto'),
    (result['Industry'] == 'Tow Truck Company'),
    (result['Industry'] == 'Tow Truck Exemption'),
    (result['Industry'] == 'Tow Truck Driver')
    
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'parking_and_automotive_services'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/3927816503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['parking_and_automotive_services'])


In [426]:
# check the rows with parking_and_automotive_services type

parking_and_automotive_services = result[result['Industry'] == 'parking_and_automotive_services']

parking_and_automotive_services

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
2415,parking_and_automotive_services,-73.996614,40.602054,2024-10-31,03/27/2017,11214,22,Brooklyn
2433,parking_and_automotive_services,-73.996614,40.602054,2024-10-31,08/13/1998,11214,22,Brooklyn
3373,parking_and_automotive_services,-73.985924,40.618792,2024-10-31,05/27/2009,11204,22,Brooklyn
3561,parking_and_automotive_services,-73.996624,40.602054,2024-10-31,01/31/1997,11214,22,Brooklyn
4948,parking_and_automotive_services,-73.996624,40.602062,2024-10-31,01/04/1996,11214,22,Brooklyn
...,...,...,...,...,...,...,...,...
28810,parking_and_automotive_services,-73.912668,40.872385,2025-03-31,04/19/2000,10034,128,Manhattan
28983,parking_and_automotive_services,-73.802841,40.865160,2025-03-31,04/29/2008,10464,184,Bronx
36608,parking_and_automotive_services,-74.014434,40.703221,2025-03-31,03/28/1997,10004,12,Manhattan
30475,parking_and_automotive_services,-73.911641,40.599915,2023-07-31,02/10/2005,11234,154,Brooklyn


In [427]:
result['Industry'] = result['Industry'].cat.add_categories(['professional_services'])

# do the assignment
conditions = [
    (result['Industry'] == 'Employment Agency'), 
    (result['Industry'] == 'Electronic & Appliance Service'), 
    (result['Industry'] == 'General Vendor Distributor'),
    (result['Industry'] == 'Process Serving Agency'),
    (result['Industry'] == 'Scale Dealer Repairer'),
    (result['Industry'] == 'Scrap Metal Processor'),
    (result['Industry'] == 'General Vendor'),
    (result['Industry'] == 'Process Server Individual')
    
    
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'professional_services'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/1293513468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['professional_services'])


In [428]:
# check the rows with professional_services type

professional_services = result[result['Industry'] == 'professional_services']

professional_services

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
31,professional_services,-73.996614,40.602054,2023-09-30,07/15/2022,11214,22,Brooklyn
277,professional_services,-74.003382,40.606545,2024-06-30,10/12/2012,11214,22,Brooklyn
1107,professional_services,-73.985921,40.618792,2023-09-30,08/16/2013,11204,22,Brooklyn
1285,professional_services,-73.995106,40.602699,2024-06-30,11/05/2021,11214,22,Brooklyn
1387,professional_services,-73.985921,40.618792,2024-02-28,02/24/2012,11204,22,Brooklyn
...,...,...,...,...,...,...,...,...
15202,professional_services,-73.986681,40.702762,2024-02-28,03/20/2019,11201,66,Brooklyn
19906,professional_services,-73.987370,40.702455,2024-06-30,11/24/2021,11201,66,Brooklyn
9289,professional_services,-74.005517,40.727392,2024-05-01,05/27/2004,10010,125,Manhattan
12395,professional_services,-74.009240,40.725209,2024-05-01,07/10/2012,10013,125,Manhattan


In [429]:
result['Industry'] = result['Industry'].cat.add_categories(['retail_services'])

# do the assignment
conditions = [
    (result['Industry'] == 'Laundry'), 
    (result['Industry'] == 'Laundries'), 
    (result['Industry'] == 'Laundry Jobber'),
    (result['Industry'] == 'General Vendor Distributor'),
    (result['Industry'] == 'Newsstand'),
    (result['Industry'] == 'Electronics Store'),
    (result['Industry'] == 'Electronic Cigarette Dealer'),
    (result['Industry'] == 'Dealer In Products'),
    (result['Industry'] == 'Secondhand Dealer - Firearms'),
    (result['Industry'] == 'Secondhand Dealer - General'),
    (result['Industry'] == 'Tobacco Retail Dealer'),
    (result['Industry'] == 'Ticket Seller Business'),
    (result['Industry'] == 'Stoop Line Stand'),
    (result['Industry'] == 'Special Sale'),
    (result['Industry'] == 'Ticket Seller')
    
    
    
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'retail_services'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/1714209455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['retail_services'])


In [430]:
# check the rows with retail_services type

retail_services = result[result['Industry'] == 'retail_services']

retail_services

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
73,retail_services,-73.993377,40.601692,2024-03-31,07/18/2022,11214,22,Brooklyn
140,retail_services,-73.999085,40.618822,2023-12-31,07/22/2020,11228,22,Brooklyn
202,retail_services,-73.993154,40.601555,2024-03-31,07/15/2022,11214,22,Brooklyn
291,retail_services,-73.991571,40.617751,2023-11-30,06/16/2018,11204,22,Brooklyn
332,retail_services,-73.990898,40.618410,2023-11-30,06/19/2018,11204,22,Brooklyn
...,...,...,...,...,...,...,...,...
31061,retail_services,-73.880000,40.773995,2024-12-31,02/28/2019,11371,138,Queens
31156,retail_services,-73.880000,40.773995,2024-12-31,10/28/2019,11371,138,Queens
33155,retail_services,-73.880000,40.773995,2024-12-31,10/05/2021,11371,138,Queens
31933,retail_services,-73.788242,40.648244,2024-12-31,07/14/2011,11430,132,Queens


In [431]:
 
result['Industry'] = result['Industry'].cat.add_categories(['financial_services'])

# do the assignment
conditions = [
    (result['Industry'] == 'Debt Collection Agency'), 
    (result['Industry'] == 'Pawnbroker')
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'financial_services'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/2741862819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['financial_services'])


In [432]:
# check the rows with financial_services type

financial_services = result[result['Industry'] == 'financial_services']

financial_services

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
17266,financial_services,-73.982229,40.774123,2024-04-30,05/11/2022,10023,142,Manhattan
9967,financial_services,-73.916409,40.815536,2024-04-30,09/02/2008,10455,168,Bronx
19162,financial_services,-73.922363,40.808895,2024-04-30,05/02/2005,10454,168,Bronx
26050,financial_services,-73.917435,40.806921,2024-04-30,06/29/1998,10454,168,Bronx
3995,financial_services,-73.808159,40.702312,2024-04-30,04/07/1997,11435,130,Queens
...,...,...,...,...,...,...,...,...
22614,financial_services,-74.008291,40.729988,2025-01-31,09/05/2018,10014,158,Manhattan
28681,financial_services,-73.984445,40.718882,2024-04-30,09/23/1997,10002,232,Manhattan
36208,financial_services,-73.985437,40.718388,2024-04-30,09/10/2008,10002,232,Manhattan
36920,financial_services,-73.987331,40.718287,2024-04-30,06/06/2022,10002,232,Manhattan


In [433]:
 
result['Industry'] = result['Industry'].cat.add_categories(['real_estate'])

# do the assignment
conditions = [
    (result['Industry'] == 'Construction Labor Provider'), 
    (result['Industry'] == 'Home Improvement Contractor'),
    (result['Industry'] == 'Commercial Lessor'),
    (result['Industry'] == 'Auction House Premises'),
    (result['Industry'] == 'Storage Warehouse'),
    (result['Industry'] == 'Locksmith'),
    (result['Industry'] == 'Locksmith Apprentice')
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'real_estate'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/3479200499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['real_estate'])


In [434]:
# check the rows with real_estate type

real_estate = result[result['Industry'] == 'real_estate']

real_estate

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
1364,real_estate,-73.993927,40.620403,2025-02-28,03/28/2023,11204,22,Brooklyn
1833,real_estate,-73.994965,40.604308,2025-02-28,07/22/2021,11214,22,Brooklyn
2580,real_estate,-73.985924,40.618792,2025-05-31,05/22/2015,11204,22,Brooklyn
2914,real_estate,-73.996624,40.602054,2025-05-31,12/14/2009,11214,22,Brooklyn
3297,real_estate,-73.990664,40.616632,2025-02-28,02/15/2017,11204,22,Brooklyn
...,...,...,...,...,...,...,...,...
38311,real_estate,-74.005617,40.726615,2025-02-28,01/09/2006,10011,125,Manhattan
17899,real_estate,-73.888945,40.770329,2025-02-28,06/21/2011,11370,138,Queens
20191,real_estate,-73.868628,40.879034,2025-02-28,06/10/2004,10467,240,Bronx
25833,real_estate,-73.825216,40.845966,2025-02-28,10/11/2022,10465,184,Bronx


In [435]:
 
result['Industry'] = result['Industry'].cat.add_categories(['transportation'])

# do the assignment
conditions = [
    (result['Industry'] == 'Horse Drawn Cab Owner'), 
    (result['Industry'] == 'Pedicab Business'),
    (result['Industry'] == 'Pedicab Driver'),
    (result['Industry'] == 'Horse Drawn Driver')
    
    
]

result.loc[np.any(conditions, axis=0), 'Industry'] = 'transportation'


/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/2157668747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].cat.add_categories(['transportation'])


In [436]:
# check the rows with transportation type

transportation = result[result['Industry'] == 'transportation']

transportation

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
731,transportation,-73.996614,40.602054,2024-04-30,08/18/2022,11214,22,Brooklyn
2660,transportation,-73.996624,40.602054,2024-04-30,03/28/2023,11214,22,Brooklyn
4065,transportation,-73.996624,40.602062,2024-04-30,06/30/2022,11214,22,Brooklyn
4432,transportation,-73.996624,40.602062,2024-04-30,03/24/2023,11214,22,Brooklyn
4643,transportation,-73.985924,40.618789,2024-04-30,03/29/2023,11204,22,Brooklyn
...,...,...,...,...,...,...,...,...
26905,transportation,-73.749655,40.673146,2024-04-30,07/21/2016,11413,139,Queens
3030,transportation,-74.178514,40.546477,2024-05-31,04/30/2007,10312,84,Staten Island
15153,transportation,-74.178514,40.546495,2024-05-31,12/02/1997,10312,84,Staten Island
15771,transportation,-74.168262,40.528334,2024-03-31,04/09/2018,10312,84,Staten Island


In [437]:
print("Industry: ", result['Industry'].unique().tolist(), "\n")
print("location_id: ", result['location_id'].unique().tolist(), "\n")
print("borough: ", result['borough'].unique().tolist(), "\n")

Industry:  ['entertainment_and_recreation', 'professional_services', 'retail_services', 'transportation', 'real_estate', 'parking_and_automotive_services', 'financial_services', 'food_and_beverage'] 

location_id:  ['22', '263', '142', '168', '115', '130', '144', '79', '242', '3', '20', '169', '112', '71', '230', '87', '92', '17', '159', '29', '100', '141', '41', '167', '126', '76', '119', '116', '129', '86', '174', '127', '227', '164', '7', '210', '209', '44', '236', '225', '239', '67', '48', '218', '212', '26', '244', '14', '147', '63', '262', '108', '95', '42', '117', '193', '37', '75', '91', '197', '78', '82', '260', '243', '181', '32', '259', '188', '47', '231', '49', '6', '68', '109', '89', '161', '135', '223', '187', '133', '228', '143', '94', '198', '176', '107', '81', '145', '80', '173', '235', '185', '13', '73', '85', '156', '183', '148', '160', '172', '106', '152', '201', '36', '250', '39', '149', '83', '10', '54', '50', '226', '208', '33', '196', '206', '219', '191', '257',

In [438]:
result

,Industry,Longitude,Latitude,License Expiration Date,License Creation Date,Address ZIP,location_id,borough
0,entertainment_and_recreation,-73.996614,40.602054,2024-03-31,01/11/2022,11214,22,Brooklyn
31,professional_services,-73.996614,40.602054,2023-09-30,07/15/2022,11214,22,Brooklyn
73,retail_services,-73.993377,40.601692,2024-03-31,07/18/2022,11214,22,Brooklyn
140,retail_services,-73.999085,40.618822,2023-12-31,07/22/2020,11228,22,Brooklyn
202,retail_services,-73.993154,40.601555,2024-03-31,07/15/2022,11214,22,Brooklyn
...,...,...,...,...,...,...,...,...
36608,parking_and_automotive_services,-74.014434,40.703221,2025-03-31,03/28/1997,10004,12,Manhattan
30475,parking_and_automotive_services,-73.911641,40.599915,2023-07-31,02/10/2005,11234,154,Brooklyn
31933,retail_services,-73.788242,40.648244,2024-12-31,07/14/2011,11430,132,Queens
32085,retail_services,-73.788242,40.648244,2024-12-31,07/14/2011,11430,132,Queens


# the code below used for model

In [439]:
result = result[['Industry', 'location_id']]
result

,Industry,location_id
0,entertainment_and_recreation,22
31,professional_services,22
73,retail_services,22
140,retail_services,22
202,retail_services,22
...,...,...
36608,parking_and_automotive_services,12
30475,parking_and_automotive_services,154
31933,retail_services,132
32085,retail_services,132


In [440]:
result['Industry'] = result['Industry'].astype(str)
result['location_id'] = result['location_id'].astype(str)

/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/3569036147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Industry'] = result['Industry'].astype(str)
/var/folders/_0/5bnj4ngn31zbtd47dr9w8dfc0000gn/T/ipykernel_45061/3569036147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['location_id'] = result['location_id'].astype(str)


In [441]:
df_combined = result.groupby(['location_id', 'Industry']).size().reset_index(name='Counts')

df_combined

,location_id,Industry,Counts
0,10,financial_services,1
1,10,parking_and_automotive_services,13
2,10,professional_services,4
3,10,real_estate,58
4,10,retail_services,29
...,...,...,...
1363,98,retail_services,31
1364,98,transportation,5
1365,99,parking_and_automotive_services,1
1366,99,real_estate,2


In [442]:
# check the outcome is correct
total_count = df_combined['Counts'].sum()
total_count

36913

In [443]:
df_combined.columns = df_combined.columns.str.replace('location_id', 'taxi_zone')
df_combined.columns = df_combined.columns.str.replace('Industry', 'business_type')

In [444]:
print("Industry: ", df_combined['business_type'].unique().tolist(), "\n")
print("location_id: ", df_combined['taxi_zone'].unique().tolist(), "\n")

Industry:  ['financial_services', 'parking_and_automotive_services', 'professional_services', 'real_estate', 'retail_services', 'food_and_beverage', 'entertainment_and_recreation', 'transportation'] 

location_id:  ['10', '100', '101', '102', '106', '107', '108', '109', '11', '112', '113', '114', '115', '116', '117', '118', '119', '12', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '195', '196', '197', '198', '20', '200', '201', '202', '203', '204', '205', '206', '208', '209', '21', '210'

In [445]:
df_combined

,taxi_zone,business_type,Counts
0,10,financial_services,1
1,10,parking_and_automotive_services,13
2,10,professional_services,4
3,10,real_estate,58
4,10,retail_services,29
...,...,...,...
1363,98,retail_services,31
1364,98,transportation,5
1365,99,parking_and_automotive_services,1
1366,99,real_estate,2


In [446]:
df_combined.to_csv("basic_business_df.csv", index=False)

# Active business

In [32]:
active_business = results_df.loc[results_df['license_status'] != 'Inactive']
print("license_status: ", active_business['license_status'].unique().tolist(), "\n")

license_status:  ['Active'] 



In [33]:
active_business.shape

(444, 32)